In [1]:
# Import necessary libraries.
from scipy.fft import fft
from tkinter import filedialog, messagebox
import scipy
from scipy.signal import butter, lfilter
import numpy as np
import tkinter as tk
from scipy import fftpack
import pyttsx3

# Global variables.
global Uploaded_File_Successfully
Uploaded_File_Successfully=False
global File_Path
File_Path=None

# Audio processing parameters.
SAMPLE_RATE = 8000  # Sample rate in Hz.
CHARACTER_DURATION = 0.04  # Duration of each character in seconds.
NUMBER_SAMPLES = int(SAMPLE_RATE * CHARACTER_DURATION)
FFT_SIZE = 1024  # fft size.

# setup for the font used.
font_name = "Helvetica"
font_size = 10  
button_font = (font_name, font_size)

# Frequency ranges for each character.
FREQUENCIES = {
    'a': [100, 1100, 2500], 'b': [100, 1100, 3000], 'c': [100, 1100, 3500],
    'd': [100, 1300, 2500], 'e': [100, 1300, 3000], 'f': [100, 1300, 3500],
    'g': [100, 1500, 2500], 'h': [100, 1500, 3000], 'i': [100, 1500, 3500],
    'j': [300, 1100, 2500], 'k': [300, 1100, 3000], 'l': [300, 1100, 3500],
    'm': [300, 1300, 2500], 'n': [300, 1300, 3000], 'o': [300, 1300, 3500],
    'p': [300, 1500, 2500], 'q': [300, 1500, 3000], 'r': [300, 1500, 3500],
    's': [500, 1100, 2500], 't': [500, 1100, 3000], 'u': [500, 1100, 3500],
    'v': [500, 1300, 2500], 'w': [500, 1300, 3000], 'x': [500, 1300, 3500],
    'y': [500, 1500, 2500], 'z': [500, 1500, 3000], ' ': [500, 1500, 3500]
}

# Create a reverse mapping from frequencies to characters.
REVERSE_FREQUENCIES = {tuple(sorted(values)): key for key, values in FREQUENCIES.items()}

# Function to decode frequencies to characters.
def decode_frequencies(frequencies):
    frequencies = tuple(sorted(frequencies))
    # Return '?' for unknown frequencies.
    return REVERSE_FREQUENCIES.get(frequencies, '?')  

# Function to apply bandpass filter to data.
def bandpass_filter(data, lowcut, highcut, order=5):
    nyquist = 0.5 * SAMPLE_RATE
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

# Function to analyze audio segment with filters.
def analyze_segment_with_filters(segment):
    low=[50,1050,2450]
    high=[550,1550,3550]
    detected_frequencies = []
    for i in range(3):
        x = bandpass_filter(segment, low[i],high[i],order=1)
        detected_frequencies.append( np.argmax(abs(fftpack.fft(x, FFT_SIZE))) * (SAMPLE_RATE / FFT_SIZE) )
        detected_frequencies[i] = int(round(detected_frequencies[i] / 100) * 100)

    return detected_frequencies

# Function to decode audio file using filters.
def decode_audio_file_with_filters(file_path):
    # Read the audio file.
    x, data = scipy.io.wavfile.read(file_path)

    # Process in 40ms segments.
    segment_length = NUMBER_SAMPLES
    decoded_string = ''

    for start in range(0, len(data), segment_length):
        segment = data[start:start + segment_length]
        if len(segment) < segment_length:
            break
        frequencies = analyze_segment_with_filters(segment)
        character = decode_frequencies(frequencies)
        decoded_string += character

    return decoded_string


def analyze_segment_with_frequencies(segment):
    # Compute the Fourier transform.
    frequencies = np.fft.rfftfreq(len(segment), 1 / SAMPLE_RATE)
    magnitudes = np.abs(fft(segment))

    # Find the three highest amplitude frequencies.
    highest_freqs = sorted(zip(frequencies, magnitudes), key=lambda x: x[1], reverse=True)[:3]
    a=[freq for freq, mag in highest_freqs]
    return a

# Function to analyze audio segment using frequencies.
def decode_audio_file_with_frequencies(file_path):
    # Read the audio file.
    x,data = scipy.io.wavfile.read(file_path)
    # Normalize the data.
    data = data / np.max(np.abs(data))
    # Process in 40ms segments.
    segment_length = NUMBER_SAMPLES  # 40ms in samples.
    decoded_string = ''

    for start in range(0, len(data), segment_length):
        segment = data[start:start + segment_length]
        if len(segment) < segment_length:
            break
        frequencies = analyze_segment_with_frequencies(segment)
        character = decode_frequencies(frequencies)
        decoded_string += character

    return decoded_string

# Function to handle audio file upload.
def upload_audio_file():
    global File_Path
    File_Path = filedialog.askopenfilename(defaultextension=".wav", filetypes=[("WAV files", "*.wav")])
    if File_Path:
        global Uploaded_File_Successfully
        Uploaded_File_Successfully = True
        messagebox.showinfo("Success", f"File uploaded successfully")


# Function to decode audio file using frequencies and update GUI.
def decode_file_frequency():
    global Uploaded_File_Successfully
    if Uploaded_File_Successfully:
        decoded_string = decode_audio_file_with_frequencies(File_Path)
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, decoded_string)
    else:
        messagebox.showinfo("Error", f"Please upload a file as a (.wav)")

# Function to decode audio file using filters and update GUI.
def decode_file_filters():
    global Uploaded_File_Successfully
    if Uploaded_File_Successfully:
        decoded_string = decode_audio_file_with_filters(File_Path)
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, decoded_string)
    else:
        messagebox.showinfo("Error", f"Please upload a file as a (.wav)")

# Function to play the decoded text.
def play_decoded_text():
    decoded_text = result_text.get("1.0", tk.END).strip()
    if not decoded_text:
        messagebox.showerror("Error", "No text available to play.")
        return
    
    # Initialize the text-to-speech engine.
    engine = pyttsx3.init()
    engine.say(decoded_text)
    engine.runAndWait()

button_bg_color = "lightblue"
# Function to open a new window for decoding options.
def open_new_window():
    new_window = tk.Toplevel()
    new_window.title("Decoder Options")
    new_window.geometry("400x150")

    button1 = tk.Button(new_window, text="Decode using Frequency", command=decode_file_frequency, font=button_font, bg=button_bg_color)
    button1.pack(pady=5)

    button2 = tk.Button(new_window, text="Decode using Filter", command=decode_file_filters, font=button_font, bg=button_bg_color)
    button2.pack(pady=5)

    button3 = tk.Button(new_window, text="Play Decoded Text", command=play_decoded_text, font=button_font, bg=button_bg_color)
    button3.pack(pady=5)

if __name__ == "__main__":
    # Create the main window
    root = tk.Tk()
    root.title("Voice-Frequency Decoder")
    root.geometry("600x400")
    root.configure(bg="#f0f0f0")
    # Title label.
    title_label = tk.Label(root, text="Voice-Frequency Decoder", font=("Helvetica", 16, "bold"), bg="#f0f0f0")
    title_label.pack(pady=10)
    

    # Create a text widget to display the output result
    result_text = tk.Text(root, height=10,width=50, font=(font_name, 12), bg="#ffffff")
    result_text.pack()

    # Add file upload button
    upload_button = tk.Button(root, text="Upload Audio File (.wav)", command=upload_audio_file, font=("Arial",12), bg=button_bg_color)
    upload_button.pack()

    # Add a button to open decoding options
    run_button = tk.Button(root, text="Run Decoder", command=open_new_window, font=("Arial",12), bg=button_bg_color)
    run_button.pack()
    # Mention project creators
    creator_label = tk.Label(root, text="Project created by Faizer & Imran & Rifat",font=("Arial", 12), fg="gray", bg="#f0f0f0")
    creator_label.pack()

    root.mainloop()
